<a href="https://colab.research.google.com/github/alecseiterr/safe_city/blob/main/Anton_Shalin/Predict_fall_Custom_YOLO8m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.0/715.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 8.6 MB/s eta 0:00:00


In [ ]:
from queue import Queue
import threading
from ultralytics import YOLO
import cv2
import gdown
import gc
from IPython.display import clear_output, display
from ipywidgets import widgets, VBox, Label, ButtonStyle, Button
from PIL import Image
import numpy as np

# Настройки
confidence = 0.4 # порог обнаружения сущности
batch_size = 1  # размера пакета кадров для одновременной обработки

# URL к вашим файлам на Google Drive
# Видео
video_url = 'https://drive.google.com/uc?id=1LzLPI4VqinjY5Ej7IuROZSTxE5v8F3VM'      # Йога

# Веса модели
#weights_url = 'https://drive.google.com/uc?id=17OR8qdvfA7BJqdSSVkbsA5rW8oVNFMsC'    # yolov8_best2.pt
weights_url = 'https://drive.google.com/uc?id=11EMvxPJrNWIjwzJnrnPfKzrYeoaWfj8n'    # yolov8_best3.pt
#weights_url = 'https://drive.google.com/uc?id=17PxXpJZ7id4g7blt6uA1vX69R6u1qWTB'    # yolov8_best6.pt

# Пути к локально сохраненным файлам
local_video_path = '/content/video.mp4'
local_weights_path = '/content/yolov8m_fall.pt'

# Поток для чтения кадров
def video_reader_thread(video_path, frame_queue, stop_event):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened() and not stop_event.is_set():
        success, frame = cap.read()
        if success:
            frame_queue.put(frame)
        else:
            break
    cap.release()

# Функция для обновления изображения
def update_display(frame, inference_result):
    # Обновление метки результатом инференса
    inference_result_label.value = inference_result
    # Преобразование BGR в RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Преобразование в изображение PIL и отображение
    image = Image.fromarray(rgb_frame)
    with display_container:
        clear_output(wait=True)
        display(image)

# Поток пакетной обработки и отображения кадров
def video_processor_batch_thread(frame_queue, model, confidence, batch_size, stop_event):
    batch_frames = []
    while not stop_event.is_set() or not frame_queue.empty():
        if not frame_queue.empty():
            frame = frame_queue.get()
            batch_frames.append(frame)

            if len(batch_frames) == batch_size:
                # Обработка пакета кадров
                batch_results = model(batch_frames, conf=confidence)

                # Собираем результаты инференса в текст
                inference_results = []
                for result in batch_results:
                    annotated_frame = result.plot(show=False)
                    inference_results.append(str(result.names))
                    update_display(annotated_frame, '\n'.join(inference_results))

                batch_frames = []  # Очистка списка для следующего пакета
        else:
            gc.collect() # Очистка памяти

# Загрузка файлов на локальный ресурс
def download_file_from_drive(url, filename):
    gdown.download(url, filename, quiet=False)
    return filename

# Функция для обработки нажатия кнопки остановки
def stop_processing(b):
    stop_event.set()
    print("Остановка обработки...")

# Загрузка файлов с Google Drive
download_file_from_drive(video_url, local_video_path)
download_file_from_drive(weights_url, local_weights_path)

# Загрузка и использование модели
model = YOLO(local_weights_path)

# Создание кнопки остановки
#stop_button = widgets.Button(description="Остановить обработку")
stop_button = widgets.Button(
    description='Остановить обработку',
    #layout=Layout(width='500px', height='50px', border='solid'),
    style=ButtonStyle(button_color='lightblue', font_weight='bold')
)
stop_button.on_click(stop_processing)

# Виджет для отображения кнопки и изображений
display_container = widgets.Output()
inference_result_label = Label()

# Отображение виджетов
display(VBox([stop_button, display_container, inference_result_label]))

# Активация асинхронной обработки параллельных потоков
frame_queue = Queue()
stop_event = threading.Event()

# Создаем и запускаем потоки
reader_thread = threading.Thread(target=video_reader_thread, args=(local_video_path, frame_queue, stop_event))                            # чтение
processor_thread = threading.Thread(target=video_processor_batch_thread, args=(frame_queue, model, confidence, batch_size, stop_event))   # пакетная обработка и отображение

# Асинхронный запуск потоков загрузки и обработки/вывода
reader_thread.start()
processor_thread.start()

reader_thread.join()
processor_thread.join()


Downloading...
From: https://drive.google.com/uc?id=1LzLPI4VqinjY5Ej7IuROZSTxE5v8F3VM
To: /content/video.mp4
100%|██████████| 8.36M/8.36M [00:00<00:00, 112MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=11EMvxPJrNWIjwzJnrnPfKzrYeoaWfj8n
From (redirected): https://drive.google.com/uc?id=11EMvxPJrNWIjwzJnrnPfKzrYeoaWfj8n&confirm=t&uuid=beef9715-5c35-4e2d-b0d2-0aaa8e31befc
To: /content/yolov8m_fall.pt
100%|██████████| 52.0M/52.0M [00:00<00:00, 106MB/s]



0: 384x640 1 fall, 2074.4ms
Speed: 13.0ms preprocess, 2074.4ms inference, 27.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fall, 2454.1ms
Speed: 7.4ms preprocess, 2454.1ms inference, 13.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fall, 1976.2ms
Speed: 4.0ms preprocess, 1976.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fall, 1468.6ms
Speed: 2.2ms preprocess, 1468.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fall, 900.8ms
Speed: 2.1ms preprocess, 900.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fall, 848.2ms
Speed: 2.1ms preprocess, 848.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fall, 837.3ms
Speed: 1.9ms preprocess, 837.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 828.3ms
Speed: 1.9ms preprocess, 828.3ms inference, 0.5ms postprocess per imag